## PDI - Phuture DeFi Index - Rebalancing Data February 2024

## Configuring packages

In [16]:
import pandas as pd
import decouple
import sys
sys.path.append('../')
from methodology import MethodologyProd
from pycoingecko import CoinGeckoAPI
key =  decouple.config("CG_KEY")
cg = CoinGeckoAPI(api_key=key)

## Index Variables

In [17]:
date = '2024-02-01'
min_mcap =  2.5e8
min_weight = 0.005
max_weight = 0.3
circ_supply_threshold = 0.2
liveness_threshold = 180 # In days
liquidity_consistency = 90 # In days
coingecko_category = "decentralized-finance-defi"

# Calculating max slippage based on liquidity threshold
liq = 5*1e6
liq_os = liq / 2 # one-side liquidity 
trade_value_tiny = 1e2
trade_value = 1e5
max_slippage = (liq_os + trade_value_tiny) / (liq_os + trade_value) - 1
# round to the nearest .01
max_slippage = round(max_slippage, 2)

# Dataframes to remove
stablecoins = pd.DataFrame(cg.get_coins_markets('usd',category='stablecoins')).set_index('id')
ctokens = pd.DataFrame(cg.get_coins_markets('usd',category='compound-tokens')).set_index('id')
atokens = pd.DataFrame(cg.get_coins_markets('usd',category='aave-tokens')).set_index('id')

methodology_class = MethodologyProd(
                                date,
                                1,
                                'ethereum',
                                '0x632806BF5c8f062932Dd121244c9fbe7becb8B48',
                                'pdi_benchmark_data',
                                'pdi_liquidities',
                                min_mcap,
                                min_weight,
                                max_weight,
                                255,
                                circ_supply_threshold,
                                liveness_threshold,
                                liquidity_consistency,
                                max_slippage,
                                coingecko_category)

### Token inclusion criteria

1. **Project and token characteristics**
    1. The project must have a listing on Defi Llama.
    1. The project’s token should have been listed on CoinGecko with pricing data at least 1 year prior to the date of inclusion in the index. 
    1. The project should have a token that is native to Ethereum L1 or L2. This excludes wrapped variants, where the underlying tokens are locked on an alt-L1.
    1. The project should be a going concern, with a dedicated team actively building, supporting and maintaining the project. 
    1. No rebasing or deflationary tokens.
    1. The project must be widely considered to be building a useful protocol or product. 
    1. Projects that have circular feedback characteristics at the core of their offering will not be considered.
    1. Synthetic tokens which derive their value from external price feeds are not permissible. 
    1. The project’s token must not have the ability to pause token transfers.
    1. The project’s protocol or product must have significant usage.
   <br>
1. **Pricing requirmeents**
    1. Token should have a manipulation resistant price feed in the form of a Chainlink price feed or a Uniswap V3 TWAP oracle with a cardinality of at least 300 and a minimum liquidity level of $1mm.
    <br>
1. **Token Supply requirements**
    1. The project's token must have a circulating supply greater than 30% of the max supply. In cases where a token does not have a max supply, the minting mechanics would need to be assessed.
    <br>
1. **Liquidity Requirements**
    1. The token must be listed on a supported exchange.
    1. The token should have in aggregate at least \$5mm of onchain liquidity across Uniswap v2, Uniswap v3, Sushiswap, Balancer v1, Balancer v2, Bancor v2 and Bancor v3.
    1. The price should experience no more than 4% price impact when executing a $100k trade.
    1. The token must have shown consistent DeFi liquidity on Ethereum.
    <br>
1. **Security Requirements**
    1. The project must have been audited by smart contract security professionals with the audit report(s) publicly available. Alternatively, the protocol must have been operating long enough to create a consensus about its safety in the decentralised finance community.

## Run Methodology

In [18]:

results,liquidity = methodology_class.main(
    'ethereum',
    df_to_remove=[stablecoins,ctokens,atokens],
    add_category_assets=['balancer','liquity','sushi','convex-finance'],
    remove_category_assets=['the-graph','staked-ether','injective-protocol','chainlink','gnosis']
    )
liquidity

Dropping dydx because a price feed is unavailable


,spot price,delivery price,slippage,blockchain
id,,,,
aave,0.010480,1.051279e-02,0.003118,ethereum
liquity,0.759881,7.576622e-01,-0.002919,ethereum
rocket-pool,0.032372,3.220514e-02,-0.005145,ethereum
ribbon-finance,2.078702,2.060288e+00,-0.008859,ethereum
maker,0.000501,4.939480e-04,-0.013296,ethereum
convex-finance,0.327806,3.222737e-01,-0.016877,ethereum
havven,0.303911,2.980147e-01,-0.019401,ethereum
lido-dao,0.322732,3.160131e-01,-0.020819,ethereum
balancer,0.264910,2.591270e-01,-0.021829,ethereum


## Show Results

In [19]:
results

,symbol,market_cap,weight,weight_converted,address,blockchain_with_highest_liq
uniswap,UNI,4748767573,0.300000,77,0x1f9840a85d5af5bf1d1762f925bdaddc4201f984,ethereum
lido-dao,LDO,2805071779,0.179268,46,0x5a98fcbea516cf06857215779fd812ca3bef1b32,ethereum
maker,MKR,1858587313,0.118779,31,0x9f8f72aa9304c8b593d555f12ef6589cc3a579a2,ethereum
aave,AAVE,1392415523,0.088987,23,0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9,ethereum
havven,SNX,1073452795,0.068603,18,0xc011a73ee8576fb46f5e1c5751ca3b9fe0af2a6f,ethereum
frax-share,FXS,804570218,0.051419,14,0x3432b6a60d23ca0dfca7761b7ab56459d9c964d0,ethereum
rocket-pool,RPL,614188153,0.039252,10,0xd33526068d116ce69f19a9ee46f0bd304f21a51f,ethereum
curve-dao-token,CRV,526484277,0.033647,8,0xd533a949740bb3306d119cc777fa900ba034cd52,ethereum
1inch,1INCH,475930378,0.030416,7,0x111111111117dc0aa78b770fa6a738034120c302,ethereum
ribbon-finance,RBN,428015506,0.027354,6,0x6123b0049f904d730db3c36a31167d9d4121fa6b,ethereum


## Contract Output

In [20]:
methodology_class.output_for_contract()

0x111111111117dc0aa78b770fa6a738034120c302,0x1f9840a85d5af5bf1d1762f925bdaddc4201f984,0x3432b6a60d23ca0dfca7761b7ab56459d9c964d0,0x4e3fbd56cd56c3e72c1403e103b45db9da5b9d2b,0x5a98fcbea516cf06857215779fd812ca3bef1b32,0x6123b0049f904d730db3c36a31167d9d4121fa6b,0x6b3595068778dd592e39a122f4f5a5cf09c90fe2,0x6dea81c8171d0ba574754ef6f8b412f2ed88c54d,0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9,0x9f8f72aa9304c8b593d555f12ef6589cc3a579a2,0xc00e94cb662c3520282e6f5717214004a7f26888,0xc011a73ee8576fb46f5e1c5751ca3b9fe0af2a6f,0xd33526068d116ce69f19a9ee46f0bd304f21a51f,0xd533a949740bb3306d119cc777fa900ba034cd52
7,77,14,4,46,6,3,2,23,31,6,18,10,8
